In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import re
import csv
import pandas as pd
from collections import Counter
from tqdm import tqdm

from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()

In [ ]:
url = 'https://en.wiktionary.org/wiki/Category:Italian_idioms'
dict_of_links = {}
while True:
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html)
    for item in tqdm(soup.select("div[class='mw-category-generated'] a[href^='/wiki/']")):
        collocation = re.search('(?<=>)(.*)(?=<)', str(item)).group(0)
        link = 'https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0)
        dict_of_links[collocation] = link
    next_page_available = 0
    for i in soup.find_all('a'):
        if i.string == 'next page':
            url='https://en.wiktionary.org/'+i['href']
            next_page_available = 1
            break
    if not next_page_available:
        break
dict_of_links

In [5]:
del dict_of_links['Italian rhetorical questions']

In [6]:
len(dict_of_links)

788

In [ ]:
list_of_definitions = []
for collocation in tqdm(dict_of_links):
    try:
        collocation_info = parser.fetch(collocation, 'italian')
        if collocation_info!=[]:
            if collocation_info[0]['definitions']!=[]:
                for i in range(len(collocation_info[0]['definitions'])):
                    list_for_collocation = [collocation]
                    if 'partOfSpeech' in collocation_info[0]['definitions'][i]:
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['partOfSpeech'])
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['text'][1:])
                    else:
                        list_for_collocation.append('')
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['text'][1:])
                    list_for_collocation.append(dict_of_links[collocation])
                    list_of_definitions.append(list_for_collocation)
    except AttributeError:
        pass
list_of_definitions

In [10]:
df = pd.DataFrame(list_of_definitions, columns = ['Collocation', 'Part of speech', 'Definition', 'Link'])
df.head()

,Collocation,Part of speech,Definition,Link
0,a bocca aperta,adjective,"[(idiomatic) stunned, motionless, speechless]",https://en.wiktionary.org/wiki/a_bocca_aperta
1,a braccio,adjective,"[(idiomatic) off-the-cuff, improvised, unprepa...",https://en.wiktionary.org/wiki/a_braccio
2,a braccio,adverb,"[(idiomatic) off the cuff, in an improvised, u...",https://en.wiktionary.org/wiki/a_braccio
3,a capo,adverb,[at the beginning of a new paragraph or new line],https://en.wiktionary.org/wiki/a_capo
4,a capo,noun,[new paragraph or new line],https://en.wiktionary.org/wiki/a_capo


In [12]:
len(df)

685

In [11]:
list_of_literal_meanings = []
for collocation in tqdm(df['Collocation']):
    collocation_info = parser.fetch(collocation, 'italian')
    if collocation_info!=[]:
        if collocation_info[0]['etymology']!=[]:
            if not re.search('(?<=Literally, ")([^"]*)', collocation_info[0]['etymology']):
                list_of_literal_meanings.append('')
            else:
                list_of_literal_meanings.append(re.search('(?<=Literally, ")([^"]*)', \
                                                      collocation_info[0]['etymology']).group(0))

100%|█████████████████████████████████████████| 685/685 [03:44<00:00,  3.05it/s]


In [13]:
len(list_of_literal_meanings)

685

In [14]:
df.insert(2, 'Literal meaning', list_of_literal_meanings)
df.head()

,Collocation,Part of speech,Literal meaning,Definition,Link
0,a bocca aperta,adjective,at open mouth.,"[(idiomatic) stunned, motionless, speechless]",https://en.wiktionary.org/wiki/a_bocca_aperta
1,a braccio,adjective,,"[(idiomatic) off-the-cuff, improvised, unprepa...",https://en.wiktionary.org/wiki/a_braccio
2,a braccio,adverb,,"[(idiomatic) off the cuff, in an improvised, u...",https://en.wiktionary.org/wiki/a_braccio
3,a capo,adverb,at head.,[at the beginning of a new paragraph or new line],https://en.wiktionary.org/wiki/a_capo
4,a capo,noun,at head.,[new paragraph or new line],https://en.wiktionary.org/wiki/a_capo


In [15]:
df.to_csv("it_idioms_wLM.tsv", sep = "\t", index=False)